In [ ]:
###遗传学要素:突变 重组 选择
###突变包含 碱基突变 插入 缺失 异位
###重组包含 减数分裂中的联会重组 以及形成配子时不同染色单体的随机分配(每组随机挑一个配成一套)
###选择包含 正负选择
###选择会导致种群数量减少and基因池变单一,这意味着能行生殖行为的个体数量减少,伴随着种群增长率降低(因为被选择掉的寄了),因此将选择系数与亲本选择及增长率合并
#########又想了一下不行,亲本不生也在种群里与亲本不在种群里是两回事,还是分开算
#############选择压力设计的不好导致在适应个体中随机选择造成好性状遗传不下去,和死亡率合并了
###子代染色体在亲本1:1的基础上 由于重组/配子随机选择的影响,很可能使得子代染色体中某个亲本的含量上升,因此将一位亲本的染色体作为模板进行重组代替1比1组合后重组
#########此时重组率由重组与随机配对合并而成 考虑到重组的遗传搭便车效应,应将重组密集的整个区域都置换
#############不需要重组,重组由于是先发生,可以整合进配子碱基挑选中

##最影响收敛速度的是出生率和死亡率,出生率越高,多样性越多,收敛世代变少但时间增加,死亡率越高,收敛世代变多,单世代时间变少
####但是考虑到许多个体不参与繁衍,对种群是副作用,应该选择掉
########试了下,随机死亡和只死最低适合度区别不大,死亡率还是影响很大,因为活得越多,后代数量越多,突变率也越高

In [6]:
import numpy as np
import pandas as pd
from torch import nn
import scipy.stats as st
import random
import seaborn as sns
from matplotlib import pyplot as plt

In [84]:
individual = 140
generations = 100
genotype = 'qwertyuiopasdfghjklzxcvbnm !@#$%^&*()_+<>?:{}][,./'
target = 'i am falling in love with robot!'

In [539]:
class genetic():
    #突变类型仅包含碱基突变
    def __init__(self, individuals: int, generations: int, target: str, genotype: str, birth_rate=4e-1, death_rate=3.5e-1):
        self.individuals = individuals
        self.generation = generations
        self.target = target
        self.genotype = genotype
        self.birth = birth_rate
        self.death = death_rate
        # self.molecular_clock = molecular_clock
        # self.recombination_rate = recombination_rate
        # self.env_volume = env_volume

        self.genes = len(list(self.target))
    
    #初始化种群矩阵
    def create_population(self):
        pop = []
        for j in range(self.individuals):
            chr = []
            for i in range(self.genes):
                chr.append(random.choice(self.genotype))
            pop.append(chr)
        return np.array(pop)

    #依据适合度和选择压力过滤基因池
    def selection(self, population):
        fit = []
        for i in range(population.shape[0]):
            fit.append(self.fitness(population[i]))
        index = np.argsort(fit)[::-1]
        # index = index[:-self.evolutionary_pressure(population)]
        return population[index]

    
    #加权随机亲本
    ###改为分段加权亲本,其实是基因片段思想
    ######试了下,三片段改四片段好像不能增加收敛速度,反而由于计算量上升收敛变慢
    def weighted_parents(self, gene_pool, n_offspring):
        pct = 0.25
        nums = round(pct * self.genes)
        target = np.array(list(self.target))
        diversity = 12
        
        pt1, pt2, pt3 = gene_pool[:, : nums], gene_pool[:, nums: 2 * nums], gene_pool[:, 3 * nums:]
        tgt1, tgt2, tgt3 = target[: nums], target[nums: 2 * nums], target[3 * nums:]
        ft1, ft2, ft3 = self.fitness(pt1, tgt1), self.fitness(pt2, tgt2), self.fitness(pt3, tgt3)
        ft1 = np.argsort(ft1)[::-1][:diversity]
        ft2 = np.argsort(ft2)[::-1][:diversity]
        ft3 = np.argsort(ft3)[::-1][:diversity]
        #三个0~0.33随机数加和,如果和<0.33则选1 2段做亲本,0.33~0.66选1 3段,0.66~选2 3段
        pr_1, pr_2 = [], []
        for _ in range(n_offspring):
            count = 0
            parts = round(1/pct)
            lmt = 0.33
            for _ in range(parts):
                count += np.random.uniform(0, lmt)
            pr1, pr2, pr3 = random.choice(gene_pool[ft1]), random.choice(gene_pool[ft2]), random.choice(gene_pool[ft3])
            if count < lmt:
                pr_1.append(pr1)
                pr_2.append(pr2)
            elif count < lmt * 2:
                pr_1.append(pr1)
                pr_2.append(pr3)
            else:
                pr_1.append(pr2)
                pr_2.append(pr3)
        return pr_1, pr_2

    #两亲本随机遗传
    def mates(self, gene_pool, n_offsprings):
        pr1, pr2 = self.weighted_parents(gene_pool, n_offsprings)
        offsprings = []
        for i in range(n_offsprings):
            offspring_temp = []
            for site1, site2 in zip(pr1[i], pr2[i]):
                cross = random.random()
                if cross < 0.38:
                    offspring_temp.append(site1)
                elif cross < 0.76:
                    offspring_temp.append(site2)
                else:
                    offspring_temp.append(random.choice(self.genotype))
            offsprings.append(offspring_temp)
        
        return offsprings

    
    #位点概率hit突变率则从基因池中挑一个突变(等概率)
    # def mutation(self, chr):
    #     template = chr
    #     sites = np.random.uniform(size=self.genes) < self.molecular_clock
    #     for i, switch in enumerate(sites):
    #         if switch:
    #             template[i] = random.choice(self.genotype)
    #     return template
       
    #选择压力去掉一定比例个体
    ####去掉了就导致如果前期一些基因选不到,后期基因型过于单一导致灭绝
    # def evolutionary_pressure(self, pop):
    #     coefficient = random.uniform(0.09, 0.3)
    #     return round(pop.shape[0] * coefficient)

    # 随机死亡
    # def death_individuals(self, pop):
    #     return pop[np.random.uniform(size=pop.shape[0]) > self.death]

    # 末位淘汰
    def death_individuals(self, pop):
        return pop[:-round(pop.shape[0] * self.death)]

    #计算适合度
    def fitness(self, chr, target=None):
        if target is None:
            target = self.target
        return np.sum(np.array(list(target)) == np.array(chr)) if len(chr.shape)==1 \
                else np.sum(np.array(list(target)) == np.array(chr), axis=1)

    def forward(self):
        pop = self.create_population()
        generation_fitness = []
        check_point = []
        fit_mean = []
        for i in range(self.generation):
            pop = self.selection(pop)
            n_offspring = round(pop.shape[0] * self.birth)
            offspring = np.array(self.mates(pop, n_offspring))
            print(pop.shape, offspring.shape, i)
            pop = self.death_individuals(pop)
            # print(pop.shape, f'死人统计')
            pop = np.vstack([pop, offspring])
            if (i + 1) % 50 == 0:
                generation_fitness.append(self.fitness(pop))
                check_point.append([''.join(i) for i in pop])
        for j in generation_fitness:
            fit_mean.append(np.mean(j[:20])) #适合度最高的那20个
        # generation_fitness = np.vstack(generation_fitness) #拼不了一点,因为种群数量不一样,即pop.shape[0]不一样

        return (generation_fitness, check_point, fit_mean)

In [540]:
test = genetic(100, 150, target, genotype)

In [541]:
tstpl = test.forward()

(100, 32) (40, 32) 0
(105, 32) (42, 32) 1
(110, 32) (44, 32) 2
(116, 32) (46, 32) 3
(121, 32) (48, 32) 4
(127, 32) (51, 32) 5
(134, 32) (54, 32) 6
(141, 32) (56, 32) 7
(148, 32) (59, 32) 8
(155, 32) (62, 32) 9
(163, 32) (65, 32) 10
(171, 32) (68, 32) 11
(179, 32) (72, 32) 12
(188, 32) (75, 32) 13
(197, 32) (79, 32) 14
(207, 32) (83, 32) 15
(218, 32) (87, 32) 16
(229, 32) (92, 32) 17
(241, 32) (96, 32) 18
(253, 32) (101, 32) 19
(265, 32) (106, 32) 20
(278, 32) (111, 32) 21
(292, 32) (117, 32) 22
(307, 32) (123, 32) 23
(323, 32) (129, 32) 24
(339, 32) (136, 32) 25
(356, 32) (142, 32) 26
(373, 32) (149, 32) 27
(391, 32) (156, 32) 28
(410, 32) (164, 32) 29
(430, 32) (172, 32) 30
(452, 32) (181, 32) 31
(475, 32) (190, 32) 32
(499, 32) (200, 32) 33
(524, 32) (210, 32) 34
(551, 32) (220, 32) 35
(578, 32) (231, 32) 36
(607, 32) (243, 32) 37
(638, 32) (255, 32) 38
(670, 32) (268, 32) 39
(704, 32) (282, 32) 40
(740, 32) (296, 32) 41
(777, 32) (311, 32) 42
(816, 32) (326, 32) 43
(856, 32) (342, 3

In [549]:
tstpl[0][-1], tstpl[2], tstpl[1][-1][0]

(array([24, 24, 24, ..., 13, 10, 14]),
 [16.45, 21.15, 23.35],
 'i a+ falling in {ykp{w@th robo<!')

In [80]:
ts = np.random.uniform(size=(2,100))
np.mean(ts, axis=1), np.median(ts, axis=1)

(array([0.44406871, 0.53593375]), array([0.40943592, 0.58806657]))

In [212]:
tsa = np.random.randint(100, size=100)

In [332]:
def tstfitness(chr, target=None):
    if target is None:
        target = np.array(list('i am falling in love with robot!'))
    return np.sum(np.array(list(target)) == np.array(chr)) if len(chr.shape)==1 \
            else np.sum(np.array(list(target)) == np.array(chr), axis=1)


In [550]:
tsttgt = np.array([list('i a+ falling in {ykp{w@th robo<!'),
  list('i[kc >nl*ilbdg$gm)#u v)th rl#ok+'),
  list('c[km >al{iy{}nmx,a#$%fut( uqlot!'),
  list('tgcmvfnl{i$(}bdx&{y% ):thqk?bo.!')])

In [551]:
tstfitness(tsttgt)

array([24, 10, 10, 10])